# EfficientNetB7

This EfficientNetB7 need 600 Resolutions in size.

I follow [Sarav](https://www.kaggle.com/sauravbhuyan/315-bird-with-efficientnetb5) notebook for writing this

## Business Understanding

This notebooks simply want to classify 315 birds using EfficientNetB7

## Data Requirements

This section I will used it for import modules

In [2]:
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

## Data Collection

All data here is collecterd form this [datasets](https://www.kaggle.com/gpiosenka/100-bird-species)

## Data Understanding

This steps used for knowing more about the data

In [3]:
# Understanding the size of example
img = cv2.imread('../input/100-bird-species/images to test/3.jpg')
img.shape

In [4]:
# Visualize the example
plt.imshow(img)
plt.show()

## Data Preparation

This steps used for preparing the data. Like I said before, EfficientNetB7 needs resolutions 600

In [5]:
train_dir = "../input/100-bird-species/train"
test_dir = "../input/100-bird-species/test"
val_dir = "../input/100-bird-species/valid"

In [6]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir, label_mode='categorical',
                                                                 image_size=(600,600), batch_size=32)

test_data =  tf.keras.preprocessing.image_dataset_from_directory(test_dir, label_mode='categorical',
                                                                 image_size=(600,600), batch_size=32)

val_data =  tf.keras.preprocessing.image_dataset_from_directory(val_dir, label_mode='categorical',
                                                                image_size=(600,600), batch_size=32)

In [7]:
len(train_data.class_names)

In [8]:
labels_names = train_data.class_names

In [9]:
plt.figure(figsize=(12,12))
for image, label in train_data.take(1):
    for i in range(9):
        plt.subplot(3, 3, i+1)
        plt.imshow(image[i].numpy().astype('uint8'))
        plt.title(labels_names[tf.argmax(label[i])])
        plt.axis("off")

## Modeling

This steps include building the model

In [10]:
base_model = tf.keras.applications.EfficientNetB7(include_top=False, weights='imagenet')

In [11]:
for layer in base_model.layers[:-5]:
    base_model.trainable = False
    
inputs = tf.keras.Input(shape=(600,600,3))

x = base_model(inputs)
x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(315, activation='softmax')(x)

eff7_model = tf.keras.Model(inputs, outputs)

In [12]:
eff7_model.summary()

In [13]:
eff7_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

In [14]:
# Model fitting

eff7_model.fit(train_data,
               epochs=2,
               steps_per_epoch = len(train_data),
               validation_data = val_data,
               validation_steps = len(val_data))

## Evaluation

In [15]:
eff7_model_evaluation = eff7_model.evaluate(test_data)

In [16]:
print(f"Efficient Model Accuracy: {eff7_model_evaluation[1] * 100 : 0.2f}%")

In [17]:
# Testing model 
test_labels = test_data.class_names

len(test_labels)

## Predicting

In [19]:
plt.figure(figsize=(16,16))

for image, label in test_data.take(1):
    model_prediction = eff7_model.predict(image)
    for i in range(18):
        plt.subplot(6,3,i+1)
        plt.imshow(image[i].numpy().astype('uint8'))
        plt.title(f"Model Prediction: {test_labels[tf.argmax(tf.round(model_prediction[i]))]}\nOriginal Labels: {test_labels[tf.argmax(label[i])]}")
        plt.axis("off")

In [20]:
plt.figure(figsize=(16,16))

for image, label in test_data.take(1):
    model_prediction = eff7_model.predict(image)
    for i in range(18):
        plt.subplot(6,3,i+1)
        plt.imshow(image[i].numpy().astype('uint8'))
        plt.title(f"Model Prediction: {test_labels[tf.argmax(tf.round(model_prediction[i]))]}\nOriginal Labels: {test_labels[tf.argmax(label[i])]}")
        plt.axis("off")

## Conclusion

This was so nice EfficientNetB7, it's approved great model with only 2 epochs can classify 315 class with 98.29% accuracy